In [1]:
%pip install obd pyserial


  Preparing metadata (setup.py) ... done
  Created wheel for obd: filename=obd-0.7.2-py3-none-any.whl size=72228 sha256=801142163d62797a291d2c12a9d2a9f4df11badab8c2f095528aeb868aaea864
  Stored in directory: /Users/macbook/Library/Caches/pip/wheels/03/b0/48/08407628f5e4c75d0f26e584c301374f30d9bd45c3171edeaa
Successfully built obd

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import obd

# Replace COM3 with your port (Windows)
# Linux example: /dev/ttyUSB0
connection = obd.OBD(portstr="COM26", baudrate=38400, fast=False)

print(connection.status())


[obd.elm327] [Errno 2] could not open port COM26: [Errno 2] No such file or directory: 'COM26'
[obd.obd] Cannot load commands: No connection to car


Not Connected


In [3]:
import obd

connection = obd.OBD("COM26")

supported_commands = connection.supported_commands

for cmd in supported_commands:
    print(cmd)


b'0201': DTC Status since DTCs cleared
b'04': Clear DTCs and Freeze data
b'0906': Calibration Verification Numbers
b'0247': DTC Absolute throttle position B
b'020F': DTC Intake Air Temp
b'010B': Intake Manifold Pressure
b'0680': Supported MIDs [81-A0]
b'06A0': Supported MIDs [A1-C0]
b'0206': DTC Short Term Fuel Trim - Bank 1
b'011C': OBD Standards Compliance
b'020D': DTC Vehicle Speed
b'0107': Long Term Fuel Trim - Bank 1
b'06A5': Misfire Cylinder 4 Data
b'0204': DTC Calculated Engine Load
b'0221': DTC Distance Traveled with MIL on
b'ATI': ELM327 version string
b'0115': O2: Bank 1 - Sensor 2 Voltage
b'0121': Distance Traveled with MIL on
b'06A3': Misfire Cylinder 2 Data
b'0635': VVT Monitor Bank 1
b'0104': Calculated Engine Load
b'0902': Vehicle Identification Number
b'0207': DTC Long Term Fuel Trim - Bank 1
b'0215': DTC O2: Bank 1 - Sensor 2 Voltage
b'0220': DTC Supported PIDs [21-40]
b'0256': DTC Long term secondary O2 trim - Bank 1
b'010F': Intake Air Temp
b'0103': Fuel System Statu

In [6]:
import serial
import time

# Update COM port if needed (Windows: COM3/COM4, Linux: /dev/ttyUSB0)
ser = serial.Serial("COM26", 38400, timeout=1)

ser.write(b"ATZ\r")     # Reset adapter
time.sleep(1)
ser.write(b"ATI\r")     # Request adapter version
time.sleep(1)

response = ser.read(100)
print("ATI Version Response:")
print(response.decode(errors='ignore'))

ser.close()


ATI Version Response:
>LM327 v1.5


In [ ]:
import obd
import csv
import time
from datetime import datetime

#--------------------------------------------------------
# 1. Connect to ELM327 (update COM port if needed)
#--------------------------------------------------------
connection = obd.OBD("COM26")  # Windows example
# Linux/Mac example: obd.OBD("/dev/ttyUSB0")

if connection.status() != obd.OBDStatus.CAR_CONNECTED:
    print("❌ ECU Not Connected! Turn ignition ON and try again.")
    exit()

#--------------------------------------------------------
# 2. Get all supported commands from your ECU
#--------------------------------------------------------
supported = connection.supported_commands

#--------------------------------------------------------
# 3. Prepare CSV logging setup
#--------------------------------------------------------
filename = "hyundai_i20_obd_data.csv"
csv_file = open(filename, "w", newline="")
csv_writer = csv.writer(csv_file)

# CSV Header (PID Names)
header = ["timestamp"] + [cmd.name for cmd in supported]
csv_writer.writerow(header)

print(f"📁 Logging all data to {filename} ... Press CTRL+C to stop.\n")

#--------------------------------------------------------
# 4. Continuous Data Logging
#--------------------------------------------------------
try:
    while True:
        row = [datetime.now().strftime("%Y-%m-%d %H:%M:%S")]
        
        for cmd in supported:
            try:
                response = connection.query(cmd)
                if response.is_null():
                    row.append("NA")   # no data for that PID
                else:
                    row.append(str(response.value))
            except:
                row.append("ERR")      # if query fails

        csv_writer.writerow(row)
        print("Logged:", row[0])  # print timestamp only
        time.sleep(1)             # 1 second interval (adjust as needed)

except KeyboardInterrupt:
    print("\n🛑 Logging stopped by user.")
    csv_file.close()
    print(f"📦 Saved CSV file: {filename}")


📁 Logging all data to hyundai_i20_obd_data.csv ... Press CTRL+C to stop.

Logged: 2025-12-27 18:56:29
Logged: 2025-12-27 18:56:36
Logged: 2025-12-27 18:56:43
Logged: 2025-12-27 18:56:50
Logged: 2025-12-27 18:56:57
Logged: 2025-12-27 18:57:03
Logged: 2025-12-27 18:57:10
Logged: 2025-12-27 18:57:17
Logged: 2025-12-27 18:57:24
Logged: 2025-12-27 18:57:31
Logged: 2025-12-27 18:57:37
Logged: 2025-12-27 18:57:44
Logged: 2025-12-27 18:57:51
Logged: 2025-12-27 18:57:58
Logged: 2025-12-27 18:58:04
Logged: 2025-12-27 18:58:11
Logged: 2025-12-27 18:58:18
Logged: 2025-12-27 18:58:25
Logged: 2025-12-27 18:58:31
Logged: 2025-12-27 18:58:38
Logged: 2025-12-27 18:58:45
Logged: 2025-12-27 18:58:52
Logged: 2025-12-27 18:58:58
Logged: 2025-12-27 18:59:05
Logged: 2025-12-27 18:59:12
Logged: 2025-12-27 18:59:19
Logged: 2025-12-27 18:59:25
Logged: 2025-12-27 18:59:32
Logged: 2025-12-27 18:59:39
Logged: 2025-12-27 18:59:46
Logged: 2025-12-27 18:59:53
Logged: 2025-12-27 18:59:59
Logged: 2025-12-27 19:00:06
Lo

In [ ]:
import obd
import csv
import time
from datetime import datetime

#--------------------------------------------------------
# 1️⃣ Connect to the ELM327 Adapter (Update COM port if needed)
#--------------------------------------------------------
connection = obd.OBD("COM26")  # Windows example
# Linux example: connection = obd.OBD("/dev/ttyUSB0")

if connection.status() != obd.OBDStatus.CAR_CONNECTED:
    print("❌ ECU not connected! Turn the ignition ON and try again.")
    exit()

#--------------------------------------------------------
# 2️⃣ Get Supported PIDs from ECU
#--------------------------------------------------------
supported = connection.supported_commands

#--------------------------------------------------------
# 3️⃣ Setup CSV File for Logging
#--------------------------------------------------------
filename = "hyundai_i20_live_obd_data.csv"
csv_file = open(filename, "w", newline="")
csv_writer = csv.writer(csv_file)

header = ["timestamp"] + [cmd.name for cmd in supported]
csv_writer.writerow(header)

print("\n📡 LIVE DATA LOGGING STARTED")
print("📁 CSV File:", filename)
print("Press CTRL + C to stop...\n")

#--------------------------------------------------------
# 4️⃣ Log & Print Data Continuously
#--------------------------------------------------------
try:
    while True:
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        row = [timestamp]

        print("--------------------------------------------------")
        print(f"⏱ Timestamp: {timestamp}")

        for cmd in supported:
            response = connection.query(cmd)

            if response.is_null():
                value = "NA"
            else:
                value = str(response.value)

            row.append(value)
            
            # 🖥️ Print each live value on the terminal
            print(f"{cmd.name:30} => {value}")

        # ✍️ Save row to CSV file
        csv_writer.writerow(row)

        print("--------------------------------------------------\n")
        time.sleep(1)  # Adjust logging speed here (1 second)

except KeyboardInterrupt:
    print("\n🛑 Logging stopped by user.")
    csv_file.close()
    print(f"📦 Data saved to: {filename}")



📡 LIVE DATA LOGGING STARTED
📁 CSV File: hyundai_i20_live_obd_data.csv
Press CTRL + C to stop...

--------------------------------------------------
⏱ Timestamp: 2025-12-27 19:02:53
MONITOR_VVT_B1                 => Unknown : -20.0 degree [PASSED]
Unknown : -20.0 degree [PASSED]
Unknown : 33.0 degree [PASSED]
Unknown : 33.0 degree [PASSED]
Unknown : -18.52 degree [PASSED]
Unknown : 32.38 degree [PASSED]
MIDS_F                         => 01111000000000000000000000000000
CLEAR_DTC                      => NA
VIN                            => bytearray(b'MALBM51BLHM339839')
ELM_VOLTAGE                    => 14.1 volt
DTC_INTAKE_TEMP                => NA
MONITOR_O2_B1S2                => Rich to lean sensor threshold voltage : 649.04 millivolt [PASSED]
Lean to rich sensor threshold voltage : 649.04 millivolt [PASSED]
Rich to lean sensor switch time : 60.0 millisecond [PASSED]
Lean to rich sensor switch time : 0.0 millisecond [PASSED]
Unknown : 999.668 millivolt [PASSED]
Unknown : 238.144 mi

[obd.elm327] Failed to read port
[obd.elm327] Failed to read port
[obd.elm327] Failed to read port
[obd.elm327] Failed to read port
[obd.elm327] Failed to read port
[obd.elm327] Failed to read port
[obd.elm327] Failed to read port
[obd.elm327] Failed to read port
[obd.elm327] Failed to read port
[obd.elm327] Failed to read port
[obd.elm327] Failed to read port
[obd.elm327] Failed to read port


DTC_LONG_FUEL_TRIM_1           => NA
ENGINE_LOAD                    => NA
INTAKE_PRESSURE                => NA
DTC_PIDS_B                     => NA
PIDS_C                         => NA
DTC_PIDS_C                     => NA
STATUS                         => NA
MONITOR_O2_HEATER_B1S1         => No tests to report
GET_CURRENT_DTC                => []
MONITOR_MISFIRE_CYLINDER_1     => No tests to report
O2_B1S1                        => NA
SPEED                          => NA
MONITOR_CATALYST_B1            => No tests to report
CALIBRATION_ID                 => bytearray(b'')
MONITOR_MISFIRE_CYLINDER_3     => No tests to report
MONITOR_O2_HEATER_B1S2         => No tests to report
DTC_FUEL_STATUS                => NA
DTC_OBD_COMPLIANCE             => NA
LONG_O2_TRIM_B1                => NA
DTC_DISTANCE_W_MIL             => NA
MIDS_A                         => 
TIMING_ADVANCE                 => NA
MONITOR_MISFIRE_CYLINDER_2     => No tests to report
GET_DTC                        => []
DTC_TI

: 